# Exract Player Profile Information
   We want Player Names, Career Positions, Height in Centimeters, Weight in Pounds, College, State of Birth and it Latitude and Longitude,
   
   1) We will merge the players' career & season cvs we download from Kaggle.com
   
   2) We will create relative dataframes(PP, P-college, & player_htwt) to insure we collect the most and accurate information.
   
   3) We will clean out players with missing relative data 

In [919]:
#dependencies
import numpy as np
import pandas as pd
import os
import csv

In [920]:
# We Import NBA PLayer's Career Profile
#    This will tell us where they are from

players = './resources/Players.csv'
career= pd.read_csv(players)

# print('Number of Rows', len(career))
career.count()

Unnamed: 0     3922
Player         3921
height         3921
weight         3921
collage        3573
born           3921
birth_city     3452
birth_state    3439
dtype: int64

In [921]:
#  We Import NBA Player's Season Profiles
#    This will tell us where they played

player_data = './resources/player_data.csv'
plyr_season = pd.read_csv(player_data)

# Renames the Column 'name' to 'Player'
plyr_season = plyr_season.rename(columns={'name': 'Player'})

# print('Number of Rows', len(plyr_season))
plyr_season.count()

Player        4550
year_start    4550
year_end      4550
position      4549
height        4549
weight        4544
birth_date    4519
college       4248
dtype: int64

In [922]:
# Merge the plyr_season & career Dataframes on 'Players'

Player_Profile_Dirty = pd.merge(plyr_season, career, how='left', on=["Player","Player"])

print('Number of Rows', len(Player_Profile_Dirty))
Player_Profile_Dirty.count()

Number of Rows 4550


Player         4550
year_start     4550
year_end       4550
position       4549
height_x       4549
weight_x       4544
birth_date     4519
college        4248
Unnamed: 0     3814
height_y       3814
weight_y       3814
collage        3497
born           3814
birth_city     3367
birth_state    3353
dtype: int64

In [947]:
# Create DataFrames and edit Column Names


new_columns = [
'Player',
'birth_state',
'position']
PP= Player_Profile_Dirty[new_columns]
# Renames Columns
PPD = PP.rename(columns={
    'birth_state':'State',})
print('length of PPD', len(PPD))



# -----------
htwt = [
    'Player',
    'height_y',
    'weight_x']
player_htwt= Player_Profile_Dirty[htwt]
# Renames Columns
P_htwt = player_htwt.rename(columns={
    'height_y':'Height_cm',
    'weight_x':'Weight_lbs'})
print('length of P_htwt', len(P_htwt))



# -----------
college = [
    'Player',
    'college']
player_college= Player_Profile_Dirty[college]
# Renames Columns
P_college = player_college.rename(columns={
   'college':'College'})
print('length of P_college', len(P_college))



# PPD.count()
# P_htwt.count()
# P_college.count()

length of PPD 4550
length of P_htwt 4550
length of P_college 4550


In [948]:
# We Import United States latitude and Longitude

LL= './resources/50_SLat_Lng.csv'
ltlg= pd.read_csv(LL)

# Renames Colums
lat_lng= ltlg.rename(columns={
    'Place Name':'State'})
lat_lng

,State,Latitude,Longitude
0,Wisconsin,44.500000,-89.500000
1,West Virginia,39.000000,-80.500000
2,Vermont,44.000000,-72.699997
3,Texas,31.000000,-100.000000
4,South Dakota,44.500000,-100.000000
5,Rhode Island,41.700001,-71.500000
6,Oregon,44.000000,-120.500000
7,New York,43.000000,-75.000000
8,New Hampshire,44.000000,-71.500000
9,Nebraska,41.500000,-100.000000


In [949]:
#  Merge Player Profile Dataframe  with Lat. & Lng Dataframe

Player_Profile_State = pd.merge(PPD,lat_lng, how='left', on=["State","State"])
# print(len(Player_Profile_State))
Player_Profile_State.count()

Player       4550
State        3353
position     4549
Latitude     2928
Longitude    2928
dtype: int64

In [950]:
# Remove Players not Born in the USA or info not recorded (1622)
# Remove Duplicate Players by -> name and state

Player_State = Player_Profile_State.dropna(axis = 0, how ='any') 


print("Old data frame length:", len(Player_Profile_State),
      "\nNew data frame length:", len(Player_State),
      "\nNumber of Player Not Born In USA: ",(len(Player_Profile_State)-len(Player_State)))
# Player_Profile_State.count()
Player_State.count()

Old data frame length: 4550 
New data frame length: 2927 
Number of Player Not Born In USA:  1623


Player       2927
State        2927
position     2927
Latitude     2927
Longitude    2927
dtype: int64

In [951]:
# ADD USA BORN PLAYERS  College Height and Weight
# Remove Duplicate Players by -> name and state
# Remove player with imcomplete Ht & Wt Data


Player_htwt = pd.merge(Player_State, P_htwt, how='left', on=["Player","Player"])

Player_htwt.drop_duplicates(subset =('Player', 'State'),
                            keep = False, inplace = True)
Player_Profile = Player_htwt.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(Player_State),
      "\nNew data frame length:", len(Player_Profile),
      "\nNumber of Players with Incomplete Ht&Wt Data: ",len(Player_State)-len(Player_Profile))

Player_Profile.count()

Old data frame length: 2927 
New data frame length: 2844 
Number of Players with Incomplete Ht&Wt Data:  83


Player        2844
State         2844
position      2844
Latitude      2844
Longitude     2844
Height_cm     2844
Weight_lbs    2844
dtype: int64

In [952]:
# Time to Add Statistics !!!



# We Import NBA Teams Location

tll = './resources/NBATeamLL.csv'
Team_LL= pd.read_csv(tll)
Team_LL

,Team,Team Name,Team Lng,Team Lat
0,BOS,Boston Celtics,-71.062228,42.366303
1,BRK,Brooklyn Nets,-73.974689,40.682650
2,NJN,Brooklyn Nets,-73.974689,40.682650
3,NYK,New York Knicks,-73.993611,40.750556
4,PHI,Philadelphia 76ers,-75.171944,39.901111
5,TOR,Toronto Raptors,-79.379167,43.643333
6,CHI,Chicago Bulls,-87.674167,41.880556
7,CLE,Cleveland Cavaliers,-81.688056,41.496389
8,DET,Detroit Pistons,-83.245556,42.696944
9,IND,Indiana Pacers,-86.155556,39.763889


In [953]:
# # We Import NBA Player Season Stats

Season_stats = './resources/Seasons_Stats.csv'
stats_season = pd.read_csv(Season_stats)
#  Renames Columns

dirty_stats = stats_season.rename(columns={
    'Tm': 'Team',
    'Pos':'Position (Season)',
    'TRB':'Rebounds',
    'STL':'Steals',
    'BLK':'Blocks',
    'AST':'Assists',
    'PTS':'Points',
    'PER':'Player Rating'})
# dirty_stats

In [954]:
season_stats = pd.merge(dirty_stats, Team_LL, how='left', on=["Team","Team"])
season_stats.count()

Unnamed: 0           24691
Year                 24624
Player               24624
Position (Season)    24624
Age                  24616
Team                 24624
G                    24624
GS                   18233
MP                   24138
Player Rating        24101
TS%                  24538
3PAr                 18839
FTr                  24525
ORB%                 20792
DRB%                 20792
TRB%                 21571
AST%                 22555
STL%                 20792
BLK%                 20792
TOV%                 19582
USG%                 19640
blanl                    0
OWS                  24585
DWS                  24585
WS                   24585
WS/48                24101
blank2                   0
OBPM                 20797
DBPM                 20797
BPM                  20797
VORP                 20797
FG                   24624
FGA                  24624
FG%                  24525
3P                   18927
3PA                  18927
3P%                  15416
2

In [955]:
# Select only the Columns that are needed.

new_columns = [
    'Year',
    'Player',
    'Team',
    'Team Name',
    'Team Lat',
    'Team Lng',
    'Rebounds',
    'Steals',
    'Blocks',
    'Assists',
    'Points',
    'Player Rating']
stats_col= season_stats[new_columns]
# stats_col

In [956]:
# Select only the 2010-2015 Seaason statistic

STATS = stats_col.loc[(stats_col['Year']>=2010)&(stats_col['Year']<=2015)]
STATS.count()

Year             3589
Player           3589
Team             3589
Team Name        3572
Team Lat         3572
Team Lng         3572
Rebounds         3589
Steals           3589
Blocks           3589
Assists          3589
Points           3589
Player Rating    3587
dtype: int64

In [973]:
# Merge the Clean PLayer Profile with the Clean Player Stats

NBA = pd.merge(STATS,Player_Profile, how='left', on=["Player","Player"])
NBA = NBA.dropna(axis = 0, how ='any')
NBA.count()

Year             2672
Player           2672
Team             2672
Team Name        2672
Team Lat         2672
Team Lng         2672
Rebounds         2672
Steals           2672
Blocks           2672
Assists          2672
Points           2672
Player Rating    2672
State            2672
position         2672
Latitude         2672
Longitude        2672
Height_cm        2672
Weight_lbs       2672
dtype: int64

In [979]:
# Add the Players Colleges if they attended

NBAProfile = pd.merge(NBA, P_college, how='left', on=["Player","Player"])
NBAProfile['MapPt'] = pd.Series(3, index=NBAProfile.index)
# NBAProfile.count()

In [959]:
NBAProfile.set_index("Player", inplace=True)

In [1005]:
import pandas as pd
from sqlalchemy import create_engine

In [1011]:
connection_string = "postgres:postgres@localhost:5432/NBA"
engine = create_engine(f'postgresql://{connection_string}')

In [1020]:
# Confirm tables
engine.table_names()

['NBAProfile']

In [1013]:
NBAProfile.head(2)

,Year,Player,Team,Team Name,Team Lat,Team Lng,Rebounds,Steals,Blocks,Assists,Points,Player Rating,State,position,Latitude,Longitude,Height_cm,Weight_lbs,College,MapPt
0,2010.0,Arron Afflalo,DEN,Denver Nuggets,39.748611,-105.007500,252.0,46.0,30.0,138.0,724.0,10.9,California,G,36.778259,-119.417931,196.0,210.0,"University of California, Los Angeles",3
1,2010.0,LaMarcus Aldridge,POR,Portland Trail Blazers,45.531667,-122.666667,627.0,67.0,48.0,160.0,1393.0,18.2,Texas,F-C,31.000000,-100.000000,211.0,260.0,University of Texas at Austin,3


In [1009]:
# NBAProfile.to_sql(name='NBAPROFILE', con=engine, if_exists='replace')

In [1021]:
NBAProfile.to_sql(name='nbaprofile', con=engine, if_exists='replace', index=True )
engine.execute("SELECT * FROM nbaprofile").fetchall()

[(0, 2010.0, 'Arron Afflalo', 'DEN', 'Denver Nuggets', 39.748611, -105.0075, 252.0, 46.0, 30.0, 138.0, 724.0, 10.9, 'California', 'G', 36.778259000000006, -119.41793100000001, 196.0, 210.0, 'University of California, Los Angeles', 3),
 (1, 2010.0, 'LaMarcus Aldridge', 'POR', 'Portland Trail Blazers', 45.531667, -122.66666699999999, 627.0, 67.0, 48.0, 160.0, 1393.0, 18.2, 'Texas', 'F-C', 31.0, -100.0, 211.0, 260.0, 'University of Texas at Austin', 3),
 (2, 2010.0, 'Malik Allen', 'DEN', 'Denver Nuggets', 39.748611, -105.0075, 82.0, 11.0, 5.0, 16.0, 105.0, 5.9, 'New Jersey', 'F-C', 39.833851, -74.871826, 208.0, 255.0, 'Villanova University', 3),
 (3, 2010.0, 'Ray Allen', 'BOS', 'Boston Celtics', 42.366303, -71.062228, 253.0, 64.0, 25.0, 210.0, 1304.0, 15.2, 'Florida', 'G', 27.994402, -81.760254, 203.0, 205.0, 'University of Connecticut', 3),
 (4, 2010.0, 'Tony Allen', 'BOS', 'Boston Celtics', 42.366303, -71.062228, 145.0, 59.0, 19.0, 72.0, 330.0, 14.2, 'Illinois', 'G-F', 40.0, -89.0, 193.

In [785]:
import gmaps
from api_keys import g_key

In [786]:
gmaps.configure(api_key=g_key)

In [831]:
birthlocations = NBAProfile[["Latitude", "Longitude"]]
Teamlocations = NBAProfile[["Team Lat", "Team Lng"]]
MP = NBAProfile["MapPt"].astype(float)

In [880]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layerB = gmaps.heatmap_layer(birthlocations, weights=(NBAProfile['Player Rating']*NBAProfile['Player Rating']), 
                                 dissipating=False, max_intensity=20000,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layerB)

# Display figure
# print(Player_Profile['State'].value_counts())
fig

Figure(layout=FigureLayout(height='420px'))

In [793]:
# Plot Heatmap
fig2 = gmaps.figure()

# Create heat layer
heat_layerT = gmaps.heatmap_layer(Teamlocations, weights=MP, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=1)
# Add layer
fig2.add_layer(heat_layerT)

# Display figure
# print(Player_Profile['State'].value_counts())
fig2

Figure(layout=FigureLayout(height='420px'))